In [154]:
import geojson
from geojson import Feature, Point, FeatureCollection
import urllib2
import json
import sys
import time
import math
import datetime
import geopy

In [155]:
def get_api_key():
    return "YOUR-API-KEY"

In [156]:
weather_station_historic_attributes = ['tempm', 'tempi', 'dewptm', 'dewpti', 'hum', 'wspdm', 'wspdi', 'wgustm', 'wgusti', 'wdird', 'wdire', 'pressurem', 'pressurei', 'windchillm', 'windchilli', 'heatindexm', 'heatindexi']

In [157]:
def get_url_from_lat_lon(latitude, longitude):
    url = 'http://api.wunderground.com/api/'+ get_api_key() + '/geolookup/q/' + str(latitude) + ',' + str(longitude) + '.json'
    return url

In [158]:
def get_neighbors(url):
    """ get the list of nearby airport and pws weather stations """
    f = urllib2.urlopen(url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    airports = parsed_json['location']['nearby_weather_stations']['airport']['station']
    icaos = map(lambda x: ("http://api.wunderground.com/api/" + get_api_key() + "/conditions/q/" + x['icao'] + ".json"), airports)
    print icaos
    pws = parsed_json['location']['nearby_weather_stations']['pws']['station']
    ids = map(lambda x: ("http://api.wunderground.com/api/" + get_api_key() + "/conditions/q/pws:" + x['id'] + '.json'), pws)
    stations = icaos + ids
    #print 'stations: '
    print stations
    return stations

In [159]:
def get_feature(station_url):
    time.sleep(20)
    print station_url
    f = urllib2.urlopen(station_url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    lat = float(parsed_json['current_observation']['display_location']['latitude'] )
    lon = float(parsed_json['current_observation']['display_location']['longitude'] )
    zip = parsed_json['current_observation']['display_location']['zip']
    weather = parsed_json['current_observation']['weather']
    temperature_string = parsed_json['current_observation']['temperature_string']
    icon = parsed_json['current_observation']['icon_url']
    point = Point((lat, lon))
    prop  = {'weather' : weather, 'temperature_string' : temperature_string, 'icon' : icon}
    f = Feature(geometry=point, properties=prop)
    print f
    return f

In [160]:
def get_feature_collection(lat, lon):
    url = get_url_from_lat_lon(lat, lon)
    print url
    stations = get_neighbors(url)
    print stations
    
    features = []
    for station_url in stations:
        # if url in DB load otherwise call url
        
        f = get_feature(station_url)
        features.append(f)
        print f
    collection = FeatureCollection(features)
    print collection
    return collection

In [161]:
get_feature_collection(34.776,-122.395)

http://api.wunderground.com/api/452d60ad282f18c9/geolookup/q/34.776,-122.395.json
[u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/KPRB.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/KSBP.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/KSMX.json']
[u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/KPRB.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/KSBP.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/KSMX.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/pws:KCASANSI3.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/pws:KCACAMBR5.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/pws:MUCRM.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/pws:KCACAMBR16.json', u'http://api.wunderground.com/api/452d60ad282f18c9/conditions/q/pws:KCACAMBR21.json', u'http://api.wunderground.com/api/452

{"features": [{"geometry": {"coordinates": [35.638363, -120.776062], "type": "Point"}, "properties": {"icon": "partlycloudy", "temperature_string": "38.0 F (3.3 C)", "weather": "Partly Cloudy"}, "type": "Feature"}, {"geometry": {"coordinates": [35.638363, -120.776062], "type": "Point"}, "properties": {"icon": "partlycloudy", "temperature_string": "38.0 F (3.3 C)", "weather": "Partly Cloudy"}, "type": "Feature"}], "type": "FeatureCollection"}